In [1]:
import numpy as np
import tensorflow as tf
import os
import numpy as np
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("MNIST_data/", one_hot=False)
from skimage.measure import compare_ssim as ssim
from skimage.measure import compare_mse as mse
from PIL import Image
import pickle
from random import shuffle

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


In [2]:
import matplotlib.pyplot as plt

In [3]:
gauss = np.random.normal(3,0.2,(784,))

train_imgs = mnist.train.images
test_imgs = mnist.test.images

noise_train = np.array([(img+gauss) for img in train_imgs])

noise_test = np.array([(img+gauss) for img in test_imgs])

In [4]:
train_labels = mnist.train.labels
test_labels = mnist.test.labels

In [5]:
train_imgs = train_imgs.reshape(train_imgs.shape[0], 28, 28, 1)
test_imgs = test_imgs.reshape(test_imgs.shape[0], 28, 28, 1)

In [6]:
noise_train = noise_train.reshape(noise_train.shape[0], 28, 28,1 )
noise_test = noise_test.reshape(noise_test.shape[0], 28, 28, 1)

In [7]:
input_shape = (28, 28, 1)

In [8]:
train_imgs = train_imgs.astype('float32')
test_imgs = test_imgs.astype('float32')
noise_train = noise_train.astype('float32')
noise_test = noise_test.astype('float32')

In [9]:
train_imgs /= 255
test_imgs /= 255
noise_train /= 255
noise_test /= 255

In [10]:
train_labels_onehot = tf.one_hot(train_labels,depth = 10)
test_labels_onehot = tf.one_hot(test_labels,depth = 10)

In [11]:
sess = tf.Session(config=tf.ConfigProto(allow_soft_placement=True))
   
train_labels_onehot = sess.run(train_labels_onehot)
test_labels_onehot = sess.run(test_labels_onehot)

### MODEL: TASK 1 ###

In [19]:
from keras.models import Sequential
from keras.layers import Dense, Conv2D, Dropout, Flatten, MaxPooling2D
from keras.optimizers import SGD, Adadelta, Adagrad, RMSprop,Adam
from keras.utils import np_utils, generic_utils
from keras.layers import Input
from keras.callbacks import EarlyStopping, ModelCheckpoint

Using TensorFlow backend.


In [20]:
def get_model():
# Creating a Sequential Model and adding the layers
    model = Sequential()
    model.add(Conv2D(28, kernel_size=(3,3), input_shape=input_shape))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Flatten()) # Flattening the 2D arrays for fully connected layers
    model.add(Dense(128, activation=tf.nn.relu))
    model.add(Dropout(0.2))
    model.add(Dense(10,activation=tf.nn.softmax))
    adm = Adam(lr=0.0001,beta_1=0.9,beta_2=0.999,epsilon=1e-08)
    model.compile(optimizer=adm,  loss='categorical_crossentropy', metrics=['accuracy'])
    return model

In [21]:
model = get_model()

In [22]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 26, 26, 28)        280       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 13, 13, 28)        0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 4732)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 128)               605824    
_________________________________________________________________
dropout_1 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 10)                1290      
Total params: 607,394
Trainable params: 607,394
Non-trainable params: 0
_________________________________________________________________


In [23]:
earlystopper = EarlyStopping(patience=5, verbose=1)
checkpointer = ModelCheckpoint('mnist_weights.h5', verbose=1, save_best_only=True)
results = model.fit(train_imgs, train_labels_onehot,epochs=30, validation_split=0.1, batch_size=8,callbacks=[earlystopper, checkpointer])



Train on 49500 samples, validate on 5500 samples
Epoch 1/30
49500/49500 [==============================] - 19s 391us/step - loss: 1.4135 - acc: 0.5924 - val_loss: 0.5628 - val_acc: 0.8744

Epoch 00001: val_loss improved from inf to 0.56281, saving model to mnist_weights.h5
Epoch 2/30
49500/49500 [==============================] - 18s 355us/step - loss: 0.5554 - acc: 0.8364 - val_loss: 0.3386 - val_acc: 0.9100

Epoch 00002: val_loss improved from 0.56281 to 0.33865, saving model to mnist_weights.h5
Epoch 3/30
49500/49500 [==============================] - 17s 348us/step - loss: 0.4303 - acc: 0.8722 - val_loss: 0.2814 - val_acc: 0.9213

Epoch 00003: val_loss improved from 0.33865 to 0.28140, saving model to mnist_weights.h5
Epoch 4/30
49500/49500 [==============================] - 17s 349us/step - loss: 0.3738 - acc: 0.8878 - val_loss: 0.2472 - val_acc: 0.9300

Epoch 00004: val_loss improved from 0.28140 to 0.24716, saving model to mnist_weights.h5
Epoch 5/30
49500/49500 [===============

In [24]:
model.evaluate(test_imgs,test_labels_onehot)

10000/10000 [==============================] - 0s 35us/step


[0.07883858420206234, 0.9751]

## Catastrophic forgetting

In [19]:
model_2 = get_model()


model_2.summary()

model_2.load_weights('mnist_weights.h5')

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 26, 26, 28)        280       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 13, 13, 28)        0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 4732)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 128)               605824    
_________________________________________________________________
dropout_1 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 10)                1290      
Total params: 607,394
Trainable params: 607,394
Non-trainable params: 0
_________________________________________________________________


In [20]:
t1_loss, t1_acc = model_2.evaluate(test_imgs,test_labels_onehot)

10000/10000 [==============================] - 4s 387us/step


In [21]:
model_2.evaluate(noise_test,test_labels_onehot)

10000/10000 [==============================] - 0s 29us/step


[13.340987274169922, 0.0974]

In [21]:
earlystopper = EarlyStopping(patience=3, verbose=1)
checkpointer = ModelCheckpoint('mnist_weights_2.h5', verbose=1, save_best_only=True)
results = model_2.fit(noise_train, train_labels_onehot,epochs=15, validation_split=0.1, batch_size=8,callbacks=[earlystopper, checkpointer])



Train on 49500 samples, validate on 5500 samples
Epoch 1/15
49500/49500 [==============================] - 18s 355us/step - loss: 0.5742 - acc: 0.8827 - val_loss: 0.1612 - val_acc: 0.9578

Epoch 00001: val_loss improved from inf to 0.16120, saving model to mnist_weights_2.h5
Epoch 2/15
49500/49500 [==============================] - 17s 353us/step - loss: 0.2505 - acc: 0.9235 - val_loss: 0.1554 - val_acc: 0.9585

Epoch 00002: val_loss improved from 0.16120 to 0.15542, saving model to mnist_weights_2.h5
Epoch 3/15
49500/49500 [==============================] - 17s 346us/step - loss: 0.2200 - acc: 0.9315 - val_loss: 0.1361 - val_acc: 0.9618

Epoch 00003: val_loss improved from 0.15542 to 0.13606, saving model to mnist_weights_2.h5
Epoch 4/15
49500/49500 [==============================] - 17s 347us/step - loss: 0.1990 - acc: 0.9382 - val_loss: 0.1218 - val_acc: 0.9665

Epoch 00004: val_loss improved from 0.13606 to 0.12176, saving model to mnist_weights_2.h5
Epoch 5/15
49500/49500 [=======

In [22]:
model_2.evaluate(noise_test,test_labels_onehot)

10000/10000 [==============================] - 0s 36us/step


[0.07414826228781603, 0.9773]

In [23]:
model_2.evaluate(test_imgs,test_labels_onehot)


10000/10000 [==============================] - 0s 33us/step


[14.428858853149414, 0.0932]

Perviously the model performed well with around 97% on images without noise. Now the accuracy dropped to 9%